In [3]:
# Import relevant libraries
import pandas as pd

In [4]:
import json
with open("../data/levski_new.json", encoding='utf-8') as f:
    levski = json.load(f)

df = pd.DataFrame(levski)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   opponent_name   459 non-null    object
 1   levski_goals    459 non-null    int64 
 2   opponent_goals  459 non-null    int64 
 3   result          459 non-null    object
 4   coach           459 non-null    object
 5   hosted          459 non-null    bool  
 6   scorers         353 non-null    object
 7   date            459 non-null    object
 8   tournament      459 non-null    object
dtypes: bool(1), int64(2), object(6)
memory usage: 29.3+ KB


In [6]:
# OK. Scorers list can be null when no player scored.
# Translate months to digits, then convert to datetimes
months = {'януари' : '01','февруари' : '02','март' :'03','април': '04','май':'05','юни':'06','юли':'07','август':'08','септември':'09','октомври':'10','ноември':'11','декември':'12'}

levski_dt = df.copy()
levski_dt['date'] = levski_dt['date'].replace(months,regex=True) # loop + str.replace faster?
levski_dt['date'] = pd.to_datetime(levski_dt['date'], format='%d %m %Y година',dayfirst=True)
levski_dt

,opponent_name,levski_goals,opponent_goals,result,coach,hosted,scorers,date,tournament
0,ЦСКА (София),1,0,win,Ясен Петров,False,[Гара Дембеле],2010-08-01,"""А"" Футболна група"
1,Локомотив (София),3,1,win,Ясен Петров,True,"[Гара Дембеле, Гара Дембеле, Гара Дембеле, Пре...",2010-08-09,"""А"" Футболна група"
2,Черноморец Бургас (Бургас),1,2,loss,Ясен Петров,False,"[Георги Андонов, Рикардо Андре, Александър Але...",2010-08-15,"""А"" Футболна група"
3,Калиакра (Каварна),3,0,win,Ясен Петров,True,"[Антон Петров (аг), Александър Александров, Ис...",2010-08-22,"""А"" Футболна група"
4,Видима-Раковски (Севлиево),2,1,win,Ясен Петров,False,"[Гара Дембеле, Дарко Тасевски, Герасим Заков]",2010-08-30,"""А"" Футболна група"
...,...,...,...,...,...,...,...,...,...
454,Локомотив 1926 (Пловдив),2,1,win,Станимир Стоилов,True,"[Андриан Краев, Милош Петрович, Филип Кръстев]",2022-02-19,Първа професионална лига - първи етап
455,ЦСКА 1948 (София),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Велтон Фелипе]",2022-02-26,Първа професионална лига - първи етап
456,Марек 1915 (Дупница),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Димитър Костадинов]",2021-09-22,Купа на България
457,Септември (Симитли),7,0,win,Станимир Стоилов,True,"[Марин Петков, Билал Бари, Билал Бари, Георги ...",2021-10-26,Купа на България


In [7]:
# Add a column with top scorer of match
def most_frequent(List):
    if isinstance(List, list):
        return max(set(List), key = List.count)
    return None

levski_dt['top_scorer'] = levski_dt['scorers'].apply(most_frequent)

In [8]:
def count_penalties(List):
    if isinstance(List, list):
        return sum(1 for str in List if "(д)" in str)
    return 0

levski_dt['no_of_penalties'] = levski_dt['scorers'].apply(count_penalties)

pd.set_option('display.max_colwidth', None)
levski_dt[levski_dt['no_of_penalties'] > 0]

,opponent_name,levski_goals,opponent_goals,result,coach,hosted,scorers,date,tournament,top_scorer,no_of_penalties
21,Берое (Стара Загора),3,2,win,Ясен Петров,False,"[Христо Йовов, Евгени Йорданов, Александър Александров, Владислав Златинов (д), Жанвион Юлу-Матондо]",2011-04-24,"""А"" Футболна група",Евгени Йорданов,1
23,Пирин Благоевград (Благоевград),3,2,win,Ясен Петров,False,"[Иван Цветков, Жанвион Юлу-Матондо, Гара Дембеле, Христо Йовов, Иван Цветков (д)]",2011-05-04,"""А"" Футболна група",Иван Цветков,1
25,Локомотив 1936 (Пловдив),3,2,win,Ясен Петров,False,"[Базле де Карвальо, Дарко Тасевски, Гара Дембеле, Гара Дембеле, Здравко Лазаров (д)]",2011-05-14,"""А"" Футболна група",Гара Дембеле,1
69,Локомотив 1926 (Пловдив),2,3,loss,Ясен Петров,False,"[Христо Йовов, Симеон Райков, Базиле де Карвальо, Йордан Тодоров, Христо Златински (д)]",2012-05-06,"""А"" Футболна група",Йордан Тодоров,1
71,Видима-Раковски (Севлиево),6,1,win,Ясен Петров,False,"[Жозе Жуниор, Жозе Жуниор, Тони Калво, Даниел Димов, Ангел Русев (д), Жозе Жуниор, Симеон Райков]",2012-05-19,"""А"" Футболна група",Жозе Жуниор,1
76,Спартак (Търнава),2,1,win,Георги Иванов,True,"[Ладислав Томачек (д), Владимир Гаджев, Шюрд Арс]",2011-07-28,Лига Европа,Ладислав Томачек (д),1
81,Берое (Стара Загора),2,1,win,Илиан Илиев,False,"[Давид Каядо (д), Марсиньо, Базиле де Карвальо]",2012-09-16,"""А"" Футболна група",Базиле де Карвальо,1
97,Пирин (Гоце Делчев),1,1,tie,Илиан Илиев,False,"[Ромен Ели, Любомир Витанов (д)]",2013-04-10,"""А"" Футболна група",Ромен Ели,1
131,Пирин (Гоце Делчев),4,1,win,Антони Здравков,True,"[Димитър Макриев, Гари Родригес, Димитър Макриев, Костадин Башов (д), Илиян Йорданов]",2013-11-03,"""А"" Футболна група - първи етап",Димитър Макриев,1
169,Хасково 2009 (Хасково),4,1,win,Георги Иванов,False,"[Луис Педро, Валери Домовчийски, Владимир Гаджев, Антонио Салас, Емил Ангелов (д)]",2014-09-20,"""А"" Футболна група - първи етап",Валери Домовчийски,1


In [9]:
pd.reset_option('display.max_colwidth')

levski_dt.to_csv('../data/levski_matches.csv')